In [ ]:
import scrapy
import scrapy.crawler as crawler

from scrapy.linkextractors import LinkExtractor
from scrapy.crawler import CrawlerProcess
from scrapy.utils.log import configure_logging
from multiprocessing import Process, Queue
from twisted.internet import reactor

class MarineSpider(scrapy.Spider):
    name = "quotes"
    link_extractor = LinkExtractor(allow=(), deny=(), allow_domains=(), deny_domains=(), deny_extensions=None, restrict_xpaths=(), restrict_css=(), tags=('a', 'area'), attrs=('href',), canonicalize=False, unique=True, process_value=None, strip=True)

    def start_requests(self):
        urls = [
            'https://deepbluecltd.github.io/LegacyMan/data/PlatformData/PD_1.html'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        for link in self.link_extractor.extract_links(response):
            self.log(f'The url is {link.url}')
            yield scrapy.Request(link.url, callback=self.parse)

# the wrapper to make it run more times
def run_spider(spider):
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result

In [ ]:
configure_logging()

print('Crawler Start:')
run_spider(MarineSpider)